In [2]:
os.chdir('/content')

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

     |████████████████████████████████| 215.7MB 11kB/s 
     |████████████████████████████████| 204kB 43.6MB/s 


In [291]:
import bz2
from tqdm import tqdm
import os
import string
import json
from pathlib import Path
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark import SQLContext
from pyspark import StorageLevel
from pyspark.sql import Row
from collections import OrderedDict
import pyspark.sql.functions as f
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint
import nltk
import collections
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('brown')
nltk.download('averaged_perceptron_tagger')
import re
from textblob import TextBlob
import csv

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# New Section

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# os.chdir('/content/drive/My Drive/VA')
# os.listdir()

['31']

# Load data

In [0]:
# main_folder = Path('31')

# folders = os.listdir(main_folder)
# folders_processed = 0
# for folder in folders:
#   files_processed = 0
#   for file in tqdm(os.listdir(main_folder / folder)):
#     file = Path(file)
#     with bz2.open(main_folder / folder / file, mode="r") as f:
#       tweets = pd.DataFrame.from_records([json.loads(line) for line in f.readlines()])
#     if (folders_processed == 0) and (files_processed == 0):
#       tweets.to_csv('/content/tweets.csv', header=True, index=False)
#     else:
#       tweets.to_csv('/content/tweets.csv', mode='a', header=False, index=False)
#     files_processed += 1
#   folders_processed += 1

100%|██████████| 60/60 [02:24<00:00,  2.17s/it]


In [0]:
# !gzip -k /content/tweets.csv

In [0]:
# !cp /content/tweets.csv.gz "/content/drive/My Drive/VA"
!cp "/content/drive/My Drive/VA/tweets.csv.gz" /content

In [0]:
conf = SparkConf().setAppName("VA")
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '20G')
        .set('spark.driver.memory', '20G')
        .set('spark.driver.maxResultSize', '2G'))

In [0]:
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

In [11]:
tweets = sqlContext.read.option("header", "true").csv("tweets.csv.gz").cache()
tweets.show()

+-------------------------------------+--------------------+-------------------+-------------------------------------+--------------------+---------+---------------------+-------------------------+--------------------+-----------------------+-----------------------+--------------------+----+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+-----------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+---------------------+
|                           created_at|                  id|             id_str|                                 text|              source|truncated|in_reply_to_status_id|i

In [246]:
tweets_with_user\
  .filter(lambda row: ('screen_name' in user_to_json(row.user)) and (row.favorited is not None))\
  .map(lambda row: (user_to_json(row.user)['screen_name'], row.favorite_count))\
  .take(20)

[('AngunWaritta', " 'in_reply_to_user_id_str': None"),
 ('pooginglamig', " 'in_reply_to_user_id_str': None"),
 ('kawahashi4869', " 'in_reply_to_user_id_str': None"),
 ('Rightsaber', " 'in_reply_to_user_id_str': None"),
 ('ri__________n', '0.0'),
 ('Fahad94680940', '0.0'),
 ('UmaQuaseHumana', '0.0'),
 ('iceleona', '0.0'),
 ('TEACHHHH_1122', '0.0'),
 ('sperandiomau', '0.0'),
 ('starlightdeIrey', " 'in_reply_to_user_id_str': None"),
 ('mareuu_', '0.0'),
 ('fiffyaaa', '0.0'),
 ('Crvg__Iago', " 'in_reply_to_user_id_str': None"),
 ('5321v', '0.0'),
 ('fetchaplaca', " 'in_reply_to_user_id_str': None"),
 ('worldgonemad5', " 'in_reply_to_user_id_str': None"),
 ('kn17_', " 'in_reply_to_user_id_str': None"),
 ('SchoolsMailing', '0.0'),
 ('MsMoxieMalone', '0.0')]

In [0]:
def remove_not_json(user):
  try:
    user_to_json(user)
    return True
  except:
    return False

In [0]:
def user_to_json(user):
    return json.loads(user.replace("'","\"").replace('None', "null").replace('False', "false").replace("True", "true"))

# Caching some preprocessing of the tweets

In [0]:
tweets_with_user = tweets\
                .rdd\
                .filter(lambda row: (row.user is not None) and (remove_not_json(row.user)))\
                .filter(lambda row: True if isinstance(user_to_json(row.user),dict) else False)\
                .cache()

## All the english tweets with some non-null text

In [0]:
english_tweets = tweets_with_user\
                    .filter(lambda row: (row.text is not None) and (row.lang is not None) and (row.lang == 'en'))\
                    .cache()                    

## All tweets where the user has the location enabled

In [0]:
tweets_with_location = tweets_with_user\
                        .filter(lambda row: 'country_code' in user_to_json(row.user))\
                        .cache()

# Extract a small version of the tweets just for developing purposes

In [0]:
# tweets_small = sqlContext.read.option("header", "true").csv("tweets_small.csv")
# tweets_with_user_small = tweets_small\
#                 .rdd\
#                 .filter(lambda row: (row.user is not None) and (remove_not_json(row.user)))\
#                 .filter(lambda row: True if isinstance(user_to_json(row.user),dict) else False)\
#                 .cache()
# tweets_with_location_small = tweets_with_user_small\
#                         .filter(lambda row: 'country_code' in user_to_json(row.user))\
#                         .cache()

# Visualizations

1. Extract the number of tweets per location (country)

In [0]:
tweets_per_loc = tweets_with_location\
  .map(lambda row: Row(user_to_json(row.user)['country_code']))\
  .toDF(['country_code'])\
  .groupBy('country_code')\
  .count()\
  .sort(f.col('count').desc())\
  .toPandas()

In [0]:
tweets_per_loc.to_csv('tweets_per_loc.csv', index=False)

2. Count the most used words in the English tweets

In [0]:
words_count = english_tweets\
  .flatMap(lambda row: [Row(words=str(token)) for token in word_tokenizer(row.text)])\
  .toDF()\
  .groupBy('words')\
  .count()\
  .sort(f.col('count').desc())\
  .toPandas()

In [0]:
words_count.to_csv('/content/drive/My Drive/VA/files_for_viz/word_count.csv', index=False)

3. Count the hashtags per location

In [0]:
hashtag_counts = english_tweets\
                    .flatMap(lambda row: [Row(hashtag) for hashtag in extract_hashtags(row.text)])\
                    .toDF(['hashtag'])\
                    .groupBy('hashtag')\
                    .count()\
                    .sort(f.col('count').desc())\
                    .toPandas()

In [0]:
hashtag_counts.to_csv('hashtag_count.csv', index=False)

4. Users with most replies

In [0]:
users_with_most_replies = tweets_with_user\
                          .filter(lambda row: ('screen_name' in user_to_json(row.user)) and (row.in_reply_to_screen_name is not None))\
                            .map(lambda row: Row(user=row.in_reply_to_screen_name))\
                            .toDF()\
                            .groupBy('user')\
                            .count()\
                            .sort(f.col('count').desc())\
                            .toPandas()

In [0]:
users_with_most_replies.to_csv('/content/drive/My Drive/VA/files_for_viz/users_most_replies.csv', index=False)

5. Users with most tweets and retweets historically

In [0]:
users_with_best_history = tweets_with_user\
                          .filter(lambda row: ('screen_name' in user_to_json(row.user)) and ('statuses_count' in user_to_json(row.user)))\
                            .map(lambda row: Row(user=user_to_json(row.user)['screen_name'], statuses_count=user_to_json(row.user)['statuses_count']))\
                            .toDF()\
                            .sort(f.col('statuses_count').desc())\
                            .toPandas()\
                            .drop_duplicates(subset='user', keep='first')

In [299]:
users_with_best_history.to_csv('/content/drive/My Drive/VA/files_for_viz/users_most_replies.csv', index=False)

,statuses_count,user
0,48660826,akiko_lawson
1324,22524290,test5f1798
1533,6689397,ElNacionalWeb
1544,5114503,CaraotaDigital
1613,5056202,Telkomsel


6. Users with the biggest number of tweets and retweets

In [0]:
users_most_active = tweets_with_user\
                    .filter(lambda row: 'screen_name' in user_to_json(row.user))\
                    .map(lambda row: Row(user_to_json(row.user)['screen_name']))\
                    .toDF(['user'])\
                    .groupBy('user')\
                    .count()\
                    .sort(f.col('count').desc())\
                    .toPandas()

In [0]:
users_most_active.to_csv('users_most_active.csv', index=False)

7. For a given location, give the users with the biggest number of followers and their activities

In [0]:
users_followers_activities = tweets_with_user\
                                .filter(lambda row: ('screen_name' in user_to_json(row.user) and ('followers_count' in user_to_json(row.user))))\
                                .map(lambda row: Row(user=user_to_json(row.user)['screen_name'], followers=user_to_json(row.user)['followers_count']))\
                                .toDF()\
                                .groupBy('user', 'followers')\
                                .count()\
                                .sort(f.col('followers').desc(), f.col('count').desc())\
                                .toPandas()

In [0]:
users_followers_activities.to_csv('/content/drive/My Drive/VA/followers_tweets_corr.csv', index=False)

8. Cluster the content of the tweets per location

In order to cluster the tweets, we use KMeans with the tfidf matrix.

In [0]:
tweets_for_clustering = english_tweets\
                        .filter(lambda row: ('screen_name' in user_to_json(row.user) and ('followers_count' in user_to_json(row.user))))\
                        .map(lambda row: Row(user=user_to_json(row.user)['screen_name'], followers=user_to_json(row.user)['followers_count'], text=row.text))\
                        .toDF()\
                        .toPandas()

In [249]:
tweets_for_clustering.head()

,followers,text,user
0,385,#Schools heading for #pupil #data #scandal - '...,SchoolsMailing
1,5817,@alleycat579 @AaronBroadmore @LallyABrown @San...,MsMoxieMalone
2,132,New post: Six design principles for artificial...,HowardKoo_IM
3,1,@JakeBoyCook @QueerMeNow Outstanding 😎,garydaniels101
4,2,@jimdwyernyt @ava @KenBurns @sarah_l_burns You...,blair_brian2014


In [0]:
def word_tokenizer_clustering(tweet):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)#after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)#filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []#looping through conditions
    for w in word_tokens:
#check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return filtered_tweet
    #print(word_tokens)
    #print(filtered_sentence)return tweet

In [0]:
def word_tokenizer(tweet):
  tweet = re.sub(r':', '', tweet)
  tweet = re.sub(r'‚Ä¶', '', tweet)
  tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
  tweet = re.sub(r'[#@]+','', tweet)
  tweet = re.sub(r'http\S+', '', tweet)
  tweet = emoji_pattern.sub(r'', tweet)
  return [word[0] for word in nltk.pos_tag(word_tokenize(tweet)) if (word[1] == 'NN') and (len(word[0]) > 3)]

In [0]:
def cluster_sentences(sentences, nb_of_clusters=5):
            tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenizer_clustering,
                                            stop_words=stopwords.words('english'),
                                            max_df=0.9,
                                            min_df=0.1,
                                            lowercase=True)
            #builds a tf-idf matrix for the sentences
            tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
            kmeans = KMeans(n_clusters=nb_of_clusters)
            kmeans.fit(tfidf_matrix)
            clusters = collections.defaultdict(list)
            for i, label in enumerate(kmeans.labels_):
                    clusters[label].append(i)
            return dict(clusters)

In [270]:
# Return the cluster topics
# Wordcloud for each cluster
texts = tweets_for_clustering['text'].tolist()
dict_words = {}
nclusters= 9
clusters = cluster_sentences(texts, nclusters)
for cluster in range(nclusters):
  for i, index_text in enumerate(clusters[cluster]):
    if cluster in dict_words:
      dict_words[cluster] += word_tokenizer(texts[index_text])
    else:
      dict_words[cluster] = word_tokenizer(texts[index_text])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [0]:
with open('/content/drive/My Drive/VA/files_for_viz/dict_cluster_words.json', 'w') as fp:
    json.dump(dict_words, fp)

9. For a given location, give the users with the biggest number of followers per category

In [0]:
dict_users = {}

for cluster in range(nclusters):
  for i, index_text in enumerate(clusters[cluster]):
    if cluster in dict_users:
      dict_users[cluster] += [(tweets_for_clustering.iloc[index_text]['user'],int(tweets_for_clustering.iloc[index_text]['followers']))]
    else:
      dict_users[cluster] = [(tweets_for_clustering.iloc[index_text]['user'],int(tweets_for_clustering.iloc[index_text]['followers']))]

In [0]:
def take_second(t):
  return t[1]

In [0]:
for i in dict_users:
  dict_users[i] = sorted(dict_users[i], key=take_second, reverse=True)[:10]

In [0]:
with open('/content/drive/My Drive/VA/files_for_viz/dict_cluster_users_followers.json', 'w') as fp:
    json.dump(dict_users, fp)

10. Sentiment for some influencing users

In [0]:
influencers = []

for cluster in dict_users:
  for influencer in dict_users[cluster]:
    influencers.append((influencer[0],influencer[1]))

In [0]:
influencers_sentiments = []

for influencer, follower_count in influencers:
  average_polarity = english_tweets\
                      .filter(lambda row: (row.entities is not None) and (influencer in row.entities))\
                      .map(lambda row: TextBlob(row.text).sentiment.polarity)\
                      .mean()

  if average_polarity != 0:
    influencers_sentiments.append((influencer, follower_count, average_polarity))

In [0]:
with open('/content/drive/My Drive/VA/files_for_viz/user_followers_polarity.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['user','follower_count','polarity'])
    for row in influencers_sentiments:
        csv_out.writerow(row)